### Import Libraries

In [ ]:
from imutils.video import FPS
import time
import multiprocessing

import numpy as np
import collections
from collections import deque

import imutils
import dlib
import cv2

### Helper Functions for Tracker

In [ ]:
def start_tracker(bbox, label, rgb, inputQueue, outputQueue):
    # construct a dlib rectangle object from the bbox
    # coordinates and then initiate the correlation tracker
    tracker = dlib.correlation_tracker()
    rect = dlib.rectangle(bbox[0], bbox[1], bbox[2], bbox[3])
    tracker.start_track()
    
    # loop indefinitely
    while True:
        # attempt to get the next frame from the input queue
        rgb = inputQueue.get()
        
        # if there is an entry in the queue, process it
        if rgb is not None:
            # update the tracker and get the position of the tracked object
            tracker.update(rgb)
            pos = tracker.get_position()
            
            # unpack the position object
            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())
            
            # include the label & bbox coordinates to the output queue
            outputQueue.put((label, (startX, startY, endX, endY)))

### Load MobileNet Model

In [ ]:
# path to MobileNet models
prototxt = './mobilenet_ssd/MobileNetSSD_deploy.prototxt'
model = './mobilenet_ssd/MobileNetSSD_deploy.caffemodel'

In [ ]:
# initialize the list of class labels that MobileNet SSD was trained in
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", 
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

allowed_classes = ["bicycle", "bus", "car","motorbike", "person"]

#map corresponding colors to classes
colors = np.random.uniform(255, 0, size=(len(CLASSES), 3))

In [ ]:
# load model from disk
print("[INFO] Loading MobileNet SSD model...")
net = cv2.dnn.readNetFromCaffe(prototxt, model)

### Define Parameters

In [ ]:
# confidence threshold
CONF = 0.5

# initialize the frame dimensions (will be set once the first frame is read)
W, H = None, None

# initialize our queues (input queue and output queue) for each object to be tracked
inputQueues = []
outputQueues = []

In [ ]:
trackers = []
trackableObjects = {}
labels = []

In [ ]:
points = [deque(maxlen=30) for _ in range(1000)]

In [ ]:
#for counting totals
person_counter = []
vehicle_counter = []

#for counting in hours
person_counter_hour = []
vehicle_counter_hour = []

#for storing the count of the previous hours
prev_hours_person_count = 0
prev_hours_vehicle_count = 0

#dictionary for count data
#define dictionary of data
count_dict = {'Total Persons': [0], 'Total Vehicles': [0], 'Day': [0], 'Date': [0], 'Time': [0]}

### Track Using Video

In [ ]:
objectTracker = {}
objectClass = {}
currentObjectID = 0

In [ ]:
# define video path
video = './data/video/test.mp4'

# write to output?
output = False

# Initialize the video stream and output video writer
print("[INFO] Starting video stream...")
vid = cv2.VideoCapture(video)

# used to record the time when we processed last frame
prev_frame_time = 0
 
# used to record the time at which we processed current frame
new_frame_time = 0

writer = None

# start FPS throughput estimator
fps = FPS().start()
frame_count = 0
skip_frames = 10

# loop over frames in video stream
while (vid.isOpened()):
    # get the next frame from the video
    ret, frame = vid.read()
    
    # check if we have reached the end of the video
    if frame is None:
        print("Completed!")
        break
    
    new_frame_time = time.time()
    
    # Perform frame preprocesing
    # resize the frame and convert from BGR to RGB
    frame = imutils.resize(frame, width=600)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    objectIDtoDelete = []
    
    for objectID in objectTracker.keys():
        trackingQuality = objectTracker[objectID].update(frame)
        if trackingQuality < 7:
            objectIDtoDelete.append(objectID)
    
    for objectID in objectIDtoDelete:
        print('Removing object ID ' + str(objectID) + ' from list of trackers.')
        objectTracker.pop(objectID, None)
    
    # *include write to output video if necessary*
    
    # if our queue is empty then it's an indication to create our first object tracker
    # check to see if we should run a more computationally expensive object detection method to aid our tracker
    if frame_count % skip_frames == 0:
        
        # get frame dimensions and convert the frame to a blob
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 0.007843, (w,h), 127.5)
        
        # input the blob to the model and get the detections/predictions
        net.setInput(blob)
        detections = net.forward()
        
        # loop over the detections
        for i in np.arange(0, detections.shape[2]):
            # extract the confidence score associated with the prediction
            confidence = detections[0, 0, i, 2]
            
            # only include detections that pass the CONF threshold
            if confidence > CONF:
                # extract the index of the class label from the detection
                idx = int(detections[0, 0, i, 1])
                label = CLASSES[idx]
                
                # if the class label is not a person, ignore it
                if CLASSES[idx] not in allowed_classes:
                    continue
                
                # compute the (x,y) coordinates of the bounding box for the object
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype('int')
                
                center = ((startX+endX)/2, (startY+endY)/2)
        
                matchObjectID = None
        
                for objectID in objectTracker.keys():
                    trackedPosition = objectTracker[objectID].get_position()
                    
                    # unpack the position object
                    t_startX = int(trackedPosition.left())
                    t_startY = int(trackedPosition.top())
                    t_endX = int(trackedPosition.right())
                    t_endY = int(trackedPosition.bottom())
                    
                    t_center = ((t_startX+t_endX)/2, (t_startY+t_endY)/2)
                    
                    if ( (t_startX <= center[0] <= t_endX) and (t_startY <= center[1] <= t_endY) and (startX <= t_center[0] <= endX) and (startY <= t_center[1] <= endY) ):
                        matchObjectID = objectID
                
                #create new object tracker if nonexistent for an object
                if matchObjectID is None:
                    print('Creating new tracker for ID ' + str(currentObjectID))
                    
                    # construct a dlib rectangle object from the
                    # bounding box coordinates and start the correlation tracker
                    tracker = dlib.correlation_tracker()
                    rect = dlib.rectangle(startX, startY, endX, endY)
                    tracker.start_track(frame_rgb, rect)
                    
                    objectTracker[currentObjectID] = tracker
                    objectClass[currentObjectID] = label
                    
                    currentObjectID+=1 
    
    person_current_count = int(0) #detect current vehicle in specific zone
    vehicle_current_count = int(0) #detect current vehicles in specific zone
    
    #display tracked objects     
    for objectID in objectTracker.keys():
        trackedPosition = objectTracker[objectID].get_position()
        class_name = objectClass[objectID]

        # unpack the position object
        t_startX = int(trackedPosition.left())
        t_startY = int(trackedPosition.top())
        t_endX = int(trackedPosition.right())
        t_endY = int(trackedPosition.bottom())
        
        #assign colors
        color = colors[CLASSES.index(class_name)]
        
        cv2.putText(frame, "ID {}".format(objectID), (t_startX,t_startY-15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 2)
        cv2.putText(frame, class_name, (t_startX,t_endY+15), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), )
        cv2.rectangle(frame, (t_startX,t_startY), (t_endX,t_endY), color, 2)
        
        center = (int((startX + endX)/2), int((startY + endY)/2)) #get center coordinates of bounding box
        
        #for counting
        h, w, _ = frame.shape

        if center[1] <= h and center[1] >= 0 and center[0] >= 0 and center[0] <= w:
            if class_name == allowed_classes[-1]: #if detected class is a person
                if int(objectID) not in person_counter:
                    person_counter.append(int(objectID))
                person_current_count += 1
            elif class_name in allowed_classes[:-1]: #if detected class is a vehicle or bike
                if int(objectID) not in vehicle_counter:
                    vehicle_counter.append(int(objectID))
                vehicle_current_count += 1
    
    #display persons count
    person_total_count = len(set(person_counter))
    person_total_count_hour = len(set(person_counter)) - prev_hours_person_count
    cv2.putText(frame, "Current Persons in Frame: " + str(person_current_count), (20, 60),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Total Persons Detected This Hour: " + str(person_total_count_hour), (20, 80),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Total Persons Detected: " + str(person_total_count), (20, 100),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)

    #display vehicle count
    vehicle_total_count = len(set(vehicle_counter))
    vehicle_total_count_hour = len(set(vehicle_counter)) - prev_hours_vehicle_count
    cv2.putText(frame, "Current Vehicles in Frame: " + str(vehicle_current_count), (20, 140),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Total Vehicles Detected This Hour: " + str(vehicle_total_count_hour), (20, 160),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)
    cv2.putText(frame, "Total Vehicles Detected: " + str(vehicle_total_count), (20, 180),cv2.FONT_HERSHEY_COMPLEX, W / 1500, (0,255,0), 1, cv2.LINE_AA)
    
    #Save count every hour
    if ( (time.localtime(time.time()).tm_min % 1 == 0) and (time.localtime(time.time()).tm_min == 0) and (time.localtime(time.time()).tm_sec == 0) ):
        #append to dictionary
        date_time_split = current_time.split()
        count_dict['Total Persons'].append(person_total_count_hour)
        count_dict['Total Vehicles'].append(vehicle_total_count_hour)
        count_dict['Day'].append(date_time_split[0])
        count_dict['Date'].append(date_time_split[2] + " " + date_time_split[1] + " " + date_time_split[4])
        count_dict['Time'].append(date_time_split[3])

        #form dataframe
        df = pd.DataFrame(count_dict)

        #save dataframe to CSV file
        df.to_csv('Person and Vehicle Count (Per Hour).csv')
        print('Saved Count Data for ' + current_time)

        #reset total person and vehicle count
        prev_hours_person_count = person_total_count
        prev_hours_vehicle_count = vehicle_total_count
        
        time.sleep(5)
    
    #display fps
    if new_frame_time - prev_frame_time != 0:
        curr_fps = 1/(new_frame_time-prev_frame_time)
        cv2.putText(frame, "FPS: {:.2f}".format(curr_fps), (20, 40),cv2.FONT_HERSHEY_COMPLEX, W / 1000, (0,0,255), 1, cv2.LINE_AA)
    prev_frame_time = new_frame_time
    
    frame_count += 1
    
    # show the output frame
    cv2.imshow("Frame", frame)
    
    # press Q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    # update the FPS counter
    fps.update()
    
# stop the timer and display FPS information
fps.stop()
print("[INFO] Elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] Approximate FPS: {:.2f}".format(fps.fps()))

# cleanup video
vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)